In [19]:
!pip install newspaper3k
!pip install gnews
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 10.9 MB/s eta 0:00:00 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=e78e77f22e1cdd78384ec765cc90cf2b5b659afdba27d37455ae2231a4d52ef8
  Stored in directory: /home/matheus/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.1
    Uninstalling beautifulsoup4-4.11.1:
      Successfully unins

In [20]:
from gnews import GNews
from newspaper import Article
import pandas as pd
import datetime

In [21]:
google_news = GNews(language='BR', country='Brazil', period='7d')

In [25]:
result = google_news.get_news('futebol')

In [27]:
df = pd.DataFrame(result)

In [33]:
links = df['url']

In [61]:
def get_text_document(url):
    article_name = Article(url, language="pt")
    article_name.download()
    article_name.parse()
    doc = {
        'title': article_name.title,
        'text': article_name.text,
        'url': article_name.url,
        'keywords':article_name.keywords,
    }
    return doc

In [46]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts='http://localhost:9200')

In [62]:
def index_data(index_name, document):
    resp = es.index(index=index_name, document=document)
    print(resp['result'])
    return resp

In [64]:
for url in links:
    try:
        doc = get_text_document(url)
        index_data(index_name='teste-index', document=doc)
    except Exception as e:
        print(f"Erro ao processar a URL {url}: {str(e)}")

created
created
created
created
created
created
created
created
Erro ao processar a URL https://news.google.com/rss/articles/CBMiugFodHRwczovL3d3dy50ZXJyYS5jb20uYnIvZXNwb3J0ZXMvZnV0ZWJvbC9pbnRlcm5hY2lvbmFsL2Nhcmxvcy1lZHVhcmRvLWNvbWVtb3JhLW1hcmNhLWRlLTEwMC1qb2dvcy1lLXByb2pldGEtY29udGludWlkYWRlLW5vLWZ1dGVib2wtZXVyb3BldSw0MTZkNTYxZTJiYWI0NTNjMTUxZTk3NGUzZDI3NDdlM3h5OWM2d3hlLmh0bWzSAb4BaHR0cHM6Ly93d3cudGVycmEuY29tLmJyL2FtcC9lc3BvcnRlcy9mdXRlYm9sL2ludGVybmFjaW9uYWwvY2FybG9zLWVkdWFyZG8tY29tZW1vcmEtbWFyY2EtZGUtMTAwLWpvZ29zLWUtcHJvamV0YS1jb250aW51aWRhZGUtbm8tZnV0ZWJvbC1ldXJvcGV1LDQxNmQ1NjFlMmJhYjQ1M2MxNTFlOTc0ZTNkMjc0N2UzeHk5YzZ3eGUuaHRtbA?oc=5&hl=en-US&gl=US&ceid=US:en: Article `download()` failed with HTTPSConnectionPool(host='www.terra.com.br', port=443): Read timed out. (read timeout=7) on URL https://news.google.com/rss/articles/CBMiugFodHRwczovL3d3dy50ZXJyYS5jb20uYnIvZXNwb3J0ZXMvZnV0ZWJvbC9pbnRlcm5hY2lvbmFsL2Nhcmxvcy1lZHVhcmRvLWNvbWVtb3JhLW1hcmNhLWRlLTEwMC1qb2dvcy1lLXByb2pldGEtY29udGludW